In [40]:
from langchain_openai import ChatOpenAI

_llm = ChatOpenAI(
    api_key="ollama",
    model="qwen2.5:7b",
    base_url="http://192.168.10.13:60001/v1",
    # temperature=0.7
)

In [41]:
_llm.invoke("今天上海的天气如何？")

AIMessage(content='为了提供准确的上海 hoje 的天气信息，我需要检查最新的气象数据，因为气象信息会随时间而变化。您可以通过查询最新天气预报或访问如中国气象局、Weather.com 等可靠的气象网站来获取上海市今天的详细天气情况。\n\n根据当前知识的更新限制，我无法直接查询实时数据，请您自行查看气象服务平台以获取准确信息。如果有其他问题或者需要帮助解读气象信息，随时可以询问我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 35, 'total_tokens': 130}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-db9e77c1-1a94-4ec2-ab0c-ec6d72eeb2da-0', usage_metadata={'input_tokens': 35, 'output_tokens': 95, 'total_tokens': 130})

In [42]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def weather(city:Annotated[str,"被查询的城市,用中文输入"]) -> str:
    """
    用于查询输入城市今日的天气状况。
    """
    if city=="上海":
        return "上海今日有台风12级"
    else:
        return "天气晴朗，风和日丽"

In [43]:
_tools = [weather,]
_llm_with_tools = _llm.bind_tools(_tools)

In [50]:
_messages = [
    ("human","今天北京的天气如何？")    
]

In [51]:
rep = _llm_with_tools.invoke(_messages)

rep

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mgjmg53m', 'function': {'arguments': '{"city":"北京"}', 'name': 'weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 168, 'total_tokens': 213}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f74fa31-a6be-4d85-a1cb-85e545bda796-0', tool_calls=[{'name': 'weather', 'args': {'city': '北京'}, 'id': 'call_mgjmg53m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 168, 'output_tokens': 45, 'total_tokens': 213})

In [56]:
from langchain_core.messages import ToolMessage

if rep.tool_calls is not None or len(rep.tool_calls)>0:
    _messages.append(rep)
    for _tool in rep.tool_calls:
        _fun = eval(_tool["name"])
        _tool_rep = _fun.invoke(_tool["args"])
        # print(_tool_rep)
        _messages.append(_tool_rep)
# _tool_rep

_llm.invoke(_messages)
print(_messages)

[('human', '今天北京的天气如何？'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mgjmg53m', 'function': {'arguments': '{"city":"北京"}', 'name': 'weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 168, 'total_tokens': 213}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4f74fa31-a6be-4d85-a1cb-85e545bda796-0', tool_calls=[{'name': 'weather', 'args': {'city': '北京'}, 'id': 'call_mgjmg53m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 168, 'output_tokens': 45, 'total_tokens': 213}), '天气晴朗，风和日丽', AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mgjmg53m', 'function': {'arguments': '{"city":"北京"}', 'name': 'weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 168, 'total_tokens': 213}, 'model_name': 'qwen2.5:7b', 'system_finger